## Dependencias

In [ ]:
pip install undetected-chromedriver

## Funciones genericas

### Definicion de mis user agent

In [19]:
import random
import time
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import json
from selenium import webdriver

user_agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0',
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
                'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:24.0) Gecko/20100101 Firefox/24.0']
agents = len(user_agents) - 1

### Definicion de funciones para busqueda de valores

In [ ]:
def search_all_coincidences(key, path, data_json):
    coincidences = []
    data_json = get_sub_path(path, data_json)
    if isinstance(data_json, list):
        for data in data_json:
            if isinstance(data, dict) or isinstance(data, list):
                coincidences = coincidences + search_all_coincidences(key, None, data)
    else:
        for data_key in data_json:
            data = data_json[data_key]
            if key == data_key:
                coincidences.append(data)
            elif isinstance(data, dict) or isinstance(data, list):
                coincidences = coincidences + search_all_coincidences(key, None, data)
    return coincidences

def get_sub_path(path, data_json):
    if path != None:
        path = path.split('/')
        for sub_path in path:
            data_json = data_json[sub_path]
    return data_json

### Funcion para escritura de resultados

In [ ]:
def write_urls(page_source, page_cont):
    
    print("Escribiendo fichero")
    initialPos = page_source.find('window.__INITIAL_PROPS__ = JSON.parse(')
    lastPos = len(page_source)
    page_source = page_source[initialPos : lastPos]
    initialPos = 39
    lastPos = page_source.find(');</script>') - 1
    page_source = page_source[initialPos : lastPos]
    page_source=page_source.replace('\\\\\\\"', '')
    page_source=page_source.replace('\\\"', '\"')
    data_json = json.loads(page_source)

    # Prepare links
    coincidences = search_all_coincidences('url', 'initialResults/items', data_json)
    file_name = "./../generated/scrapy/urls/car_liks" + str(page_cont) + ".json"
    f = open(file_name, "a")
    f.write(json.dumps(coincidences))
    f.close()

### Funciones para scrapy

In [ ]:
def scrapy_url(driver, current_page, page_cont):
    driver.get(current_page)
    page_source = driver.page_source
    if current_page not in pages_to_avoid_robot_detection:
        write_urls(page_source, page_cont)
        page_cont = page_cont + 1
    else:
        print("Pagina sin interes: " + current_page)
    return page_cont

In [ ]:
def run_scrapy_urls(page_cont, driver, pages):
        
    for page in pages:
        print("Procesando (" + str(page_cont) + " de " + str(len(pages)) + "): " + page)
        try:
            page_cont=scrapy_url(driver, page, page_cont)
        except:
            page_source = driver.page_source
            if "Algo en tu navegador nos hizo pensar que eres un bot" in page_source:
                while True:
                    print("Esperando resolucion de captcha")
                    time.sleep(30)
                    print("Probando de nuevo")
                    try:
                        page_cont=scrapy_url(driver, page, page_cont)
                        break
                    except Exception as e:
                        print("Hubo algun problema con la url: " + page)
                        print(e)

## Parametros y creacion de lista de urls para scrapy

In [ ]:
pages_configuration_for_model = {
'1': 2, '4': 40, '27': 4, # Dani
'7': 10, '11': 6, # Maximo
'1011': 3, '14': 3,'15': 10, # David
'69': 2, '18': 5, '20': 3, '21': 3, '22': 5, # Adri
'25': 5, '28': 10, '30': 3, # Miguel
'222': 6, '31': 7, '34': 4, # Carlos
'32': 11, '40': 4, # Angel
'33': 12, '48': 4, # Fran
'35': 9, '39': 9, '46': 7, '47': 10 # ??
}

pages_configuration_for_model = {
'21': 3, '22': 5, # Adri
'25': 5, '28': 10, '30': 3, # Miguel
'222': 6, '31': 7, '34': 4, # Carlos
'32': 11, '40': 4, # Angel
'33': 12, '48': 4, # Fran
'35': 9, '39': 9, '46': 7, '47': 10 # ??
}

# pages_configuration_for_model = {
# '1': 1, '4': 1, '27': 1, # Dani
# '7': 1, '11': 1, # Maximo
# '1011': 1, '14': 1,'15': 1, # David
# '69': 1, '18': 1, '20': 1, '21': 1, '22': 1, # Adri
# '25': 1, '28': 1, '30': 1, # Miguel
# '222': 1, '31': 1, '34': 1, # Carlos
# '32': 1, '40': 1, # Angel
# '33': 1, '48': 1, # Fran
# '35': 1, '39': 1, '46': 1, '47': 1 # ??
# }

pages_to_avoid_robot_detection = [ 'https://www.coches.net', 'https://www.coches.net/segunda-mano', 'https://www.coches.net/segunda-mano/?MaxYear=2020', 'https://www.coches.net/segunda-mano/?MaxYear=2020&MinYear=2010', 'https://www.coches.net/segunda-mano/?MaxYear=2020&MinKms=10000&MinYear=2010' ]

all_url_pages = [ 'https://www.coches.net/segunda-mano/?MakeId=' + k + '&MaxYear=2020&MinKms=10000&MinYear=2010&pg=' + str(num) for k, v in pages_configuration_for_model.items() for num in range(1, v + 1)  ]

config_pages = pages_to_avoid_robot_detection + all_url_pages

## Ejecucion

### Creacion de driver chrome

In [ ]:
def create_driver_chrome():
    # Prepare navigator
    user_agent_for_page = user_agents[random.randint(0, agents)]
    options = webdriver.ChromeOptions() 
    # options.headless = True
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument(user_agent_for_page)
    driver = uc.Chrome(options=options)
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent_for_page})
    return driver

In [ ]:
current_driver = create_driver_chrome()

### Scrapy

In [ ]:
run_scrapy_urls(0, current_driver, config_pages)